In [1]:
import numpy as np
import amstrax
import strax
import matplotlib.pyplot as plt
import pandas as pd
import snappy
from pymongo import MongoClient
%matplotlib inline

In [2]:
%matplotlib inline

In [3]:
# run_id = '190516_095707'_
run_id = '190610_085915'

In [4]:
st = strax.Context(storage=strax.DataDirectory('./data'),
                  register=[amstrax.RecordsFromMongo, amstrax.PulseProcessing, amstrax.Peaks, amstrax.TriggerPeaks],
                    register_all=amstrax,
#                    config=dict(pax_raw_dir='/data/xenon/xams/run10/raw/data/'),
                  )

In [5]:
# !ls ./data

In [6]:
 ! rm -r ./data/190610_085915-records*

In [ ]:
# rr = st.get_array(run_id, 'raw_records',config=dict(collection_name = run_id))

In [ ]:
# hits = strax.find_hits(rr,threshold = 15)

In [ ]:
%%timeit
r = st.get_array(run_id, 'records')

Almost done!


In [ ]:
r[0]

In [ ]:

# def fill_record(raw_records, hits):
#     skipper = 0
#     tw = 50
#     buffer = np.zeros(100,dtype = strax.record_dtype())
#     offset = 0
#     samples_per_record = strax.DEFAULT_RECORD_LENGTH
#     tw = 50
#     skipper = 0
    
#     for i, h in enumerate(hits[15:]):
#         if skipper != 0:
#             # print(skipper)
#             skipper -= 1
#             continue

#         hit = []
#         hit.append(h)
#         h_c = hits[hits['channel'] == h['channel']]
#         for h_ in h_c:
#             if h_['time'] > hit[-1]['time'] and h_['time'] < (hit[-1]['time'] + h['dt'] * tw):
#                 hit.append(h_)
#             if h_['time'] > (hit[-1]['time'] + h['dt'] * tw):
#                 break
                
#         hit_buffer = np.zeros(len(hit), dtype=strax.hit_dtype)
#         for i in range(len(hit)):
#             hit_buffer[i] = hit[i]

#         dt = hit_buffer[0]['dt']
#         start = np.min(hit_buffer['time']) - dt * tw
#         end = np.max(hit_buffer['time']) + dt * tw
#         p_length = int((end - start) / dt)
#         records_needed = int(np.ceil((end - start) / (samples_per_record * dt)))
#         # r = np.zeros(records_needed, dtype=strax.record_dtype())

#         p_offset = hit_buffer[0]['left'] - tw
#         print(hit_buffer)
#         p_end = hit_buffer[-1]['right'] + tw
#         input_record_index = [np.unique(hit_buffer['record_i']).tolist()][0]

#         print(input_record_index)
#         if p_offset < 0:
#             p_offset += samples_per_record
#             input_record_index.append(hit_buffer[0]['record_i'] - 7)

#             print('here1', input_record_index)

#         if p_end > samples_per_record:
#             input_record_index.append(hit_buffer[-1]['record_i'] + 7)
#             print('here2', input_record_index)

#         input_record_index.sort()
#         print(p_offset,p_end)
#         print(input_record_index)
#         record_buffer = []
#         for i in input_record_index:
#             record_buffer.extend(list(raw_records[i]['data']))

#         n_store = 0
#         for rec_i in range(records_needed):
#             r_ = buffer[offset + rec_i]
#             r_['dt'] = dt
#             r_['channel'] = hit_buffer[0]['channel']
#             print(r_['channel'])
#             r_['length'] = p_length
#             r_['record_i'] = rec_i
#             r_['time'] = start + rec_i * samples_per_record * dt

#             p_offset += n_store
#             if rec_i != records_needed - 1:
#                 n_store = samples_per_record
#                 # if rec_i == 0:
#                     # n_store = samples_per_record - p_offset
#             else:
#                 n_store = p_length - samples_per_record * rec_i
#             print(offset,p_length, n_store, p_offset, len(record_buffer))
#             r_['data'][:n_store] = record_buffer[p_offset:p_offset + n_store]
#             # print(buffer[offset + rec_i])


In [ ]:
# records = fill_record(rr, hits)